In [2]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt

In [28]:
import warnings
warnings.filterwarnings("ignore")

In [29]:
df = pd.read_csv("C:/Users/officedepot/Desktop/Label.txt",sep=",,,",header=None ,names=['question','type'])

In [30]:
df.head()

,question,type
0,how did serfdom develop in and then leave russ...,unknown
1,what films featured the character popeye doyle ?,what
2,how can i find a list of celebrities ' real na...,unknown
3,what fowl grabs the spotlight after the chines...,what
4,what is the full form of .com ?,what


In [31]:
df.shape

(1483, 2)

In [32]:
df['type']=df['type'].str.strip()

In [33]:
df['type'].unique()

array(['unknown', 'what', 'when', 'who', 'affirmation'], dtype=object)

In [34]:
df['question'].values

array(['how did serfdom develop in and then leave russia ? ',
       'what films featured the character popeye doyle ? ',
       "how can i find a list of celebrities ' real names ? ", ...,
       'does this hose have one ? ', 'can i get it in india ? ',
       'would this work on a 2008 ford edge with a naked roof ? '],
      dtype=object)

In [88]:
df['question'] = df['question'].apply(lambda x: x.lower())
df['question'] = df['question'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [37]:
VALIDATION_SPLIT=0.20

In [38]:
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle as pkl
from sklearn.naive_bayes import MultinomialNB
# organize imports
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [39]:
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
from nltk.corpus import wordnet as wn

In [40]:
class StemTokenizer(object):
    def __init__(self):
        self.ignore_set = {'footnote', 'nietzsche', 'plato', 'mr.'}

    def __call__(self, doc):
        words = []
        for word in word_tokenize(doc):
            word = word.lower()
            w = wn.morphy(word)
            if w and len(w) > 1 and w not in self.ignore_set:
                words.append(w)
        return words

In [41]:
stemmer = SnowballStemmer('english').stem
def stem_tokenize(text):
    return [stemmer(i) for i in word_tokenize(text)]

In [45]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\officedepot\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [46]:
vectorizer = CountVectorizer(analyzer='word',lowercase=True,tokenizer=stem_tokenize)
X_train = vectorizer.fit_transform(df.question.values)
with open('vectorizer.pk', 'wb') as fin:
    pkl.dump(vectorizer, fin)

In [89]:
labels = data['type']

In [90]:
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train = X_train[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X_train.shape[0])

x_train = X_train[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X_train[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [91]:
clf = MultinomialNB()
clf.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [92]:
preds = clf.predict(x_val)
print(classification_report(preds,y_val))
print("Accuracy :",clf.score(x_val,y_val))

              precision    recall  f1-score   support

 affirmation       0.00      0.00      0.00         0
     unknown       0.00      0.00      0.00         0
        what       0.95      0.43      0.60       284
        when       0.00      0.00      0.00         0
         who       0.04      0.25      0.07        12

    accuracy                           0.43       296
   macro avg       0.20      0.14      0.13       296
weighted avg       0.92      0.43      0.57       296

Accuracy : 0.42567567567567566


In [93]:
example=vectorizer.transform(["What time does the train leave"])
clf.predict(example)

array(['what'], dtype='<U11')

In [80]:
tf_vectorizer = TfidfVectorizer(analyzer='word',lowercase=True,tokenizer=stem_tokenize)
X_train = tf_vectorizer.fit_transform(df.question.values)
with open('tf_vectorizer.pk', 'wb') as fin:
    pkl.dump(tf_vectorizer, fin)

In [81]:
labels = data['type']
# split the data into a training set and a validation set
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train = X_train[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X_train.shape[0])

x_train = X_train[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X_train[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [82]:
clf = MultinomialNB()
clf.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [83]:
# evaluate the model of test data
preds = clf.predict(x_val)
print(classification_report(preds,y_val))
print("Accuracy :",clf.score(x_val,y_val))


              precision    recall  f1-score   support

 affirmation       0.05      1.00      0.09         1
     unknown       0.50      0.97      0.66        31
        what       1.00      0.56      0.72       206
        when       0.00      0.00      0.00         0
         who       0.66      0.91      0.77        58

    accuracy                           0.68       296
   macro avg       0.44      0.69      0.45       296
weighted avg       0.88      0.68      0.72       296

Accuracy : 0.6756756756756757


In [84]:
example=tf_vectorizer.transform(["What time does the train leave"])
clf.predict(example)

array(['what'], dtype='<U11')

In [62]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [63]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH=30

In [64]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re

In [65]:
data=df.copy()

In [66]:
print(data['type'].value_counts())

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, split=' ')
tokenizer.fit_on_texts(data['question'].values)
X = tokenizer.texts_to_sequences(data['question'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

what           609
who            402
unknown        272
affirmation    104
when            96
Name: type, dtype: int64


In [67]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Y = data['type']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
Y=le.transform(Y) 
labels = to_categorical(np.asarray(Y))
print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', labels.shape)


# split the data into a training set and a validation set
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X.shape[0])

x_train = X[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Found 3675 unique tokens.
Shape of data tensor: (1483, 30)
Shape of label tensor: (1483, 5)


In [87]:
EMBEDDING_DIM=300

In [71]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [72]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [73]:
embed_dim = 300
lstm_out = 196

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(lstm_out, dropout_U=0.25, dropout_W=0.25))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 300)           1102800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               389648    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 985       
Total params: 1,493,433
Trainable params: 390,633
Non-trainable params: 1,102,800
_________________________________________________________________
None


In [74]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          validation_data=(x_val, y_val))

Train on 1187 samples, validate on 296 samples
Epoch 1/20
1187/1187 [==============================] - 26s 22ms/step - loss: 1.5545 - accuracy: 0.3564 - val_loss: 1.5210 - val_accuracy: 0.4662
Epoch 2/20
1187/1187 [==============================] - 14s 12ms/step - loss: 1.4573 - accuracy: 0.3976 - val_loss: 1.4451 - val_accuracy: 0.4662
Epoch 3/20
1187/1187 [==============================] - 11s 9ms/step - loss: 1.4093 - accuracy: 0.3951 - val_loss: 1.4122 - val_accuracy: 0.4662
Epoch 4/20
1187/1187 [==============================] - 12s 10ms/step - loss: 1.4135 - accuracy: 0.3968 - val_loss: 1.3984 - val_accuracy: 0.4662
Epoch 5/20
1187/1187 [==============================] - 10s 9ms/step - loss: 1.4005 - accuracy: 0.3968 - val_loss: 1.4010 - val_accuracy: 0.4662
Epoch 6/20
1187/1187 [==============================] - 9s 8ms/step - loss: 1.4038 - accuracy: 0.3968 - val_loss: 1.4028 - val_accuracy: 0.4662
Epoch 7/20
1187/1187 [==============================] - 10s 8ms/step - loss: 1.40

In [111]:
example = tokenizer.texts_to_sequences(["What time does the train leave"])
example = pad_sequences(example, maxlen=MAX_SEQUENCE_LENGTH)
